____Rawan khaled____<br>
____sentiment_app____<br>

____Sentiment project predict if sentence are positive  or negative  using Python and machine learning____


_____________________

______import libraries______<br>
__load data__<br>
__assign column names__

In [4]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

nltk.download('stopwords')

df = pd.read_csv(r"C:/Users/RAWAN/OneDrive/Desktop/pipline_internship\data_11/training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None)

df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']


df = df[['sentiment', 'text']]


print(df.head())


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RAWAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   sentiment                                               text
0          0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          0  is upset that he can't update his Facebook by ...
2          0  @Kenichan I dived many times for the ball. Man...
3          0    my whole body feels itchy and like its on fire 
4          0  @nationwideclass no, it's not behaving at all....


_______build function to clean data_______

In [5]:
def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)

    text = re.sub(r'\@\w+|\#','', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text) 
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    text = text.translate(str.maketrans('', '', string.punctuation))

    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text


____apply function____

In [6]:
df['clean_text'] = df['text'].apply(clean_text)
print(df.head())

   sentiment                                               text  \
0          0  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1          0  is upset that he can't update his Facebook by ...   
2          0  @Kenichan I dived many times for the ball. Man...   
3          0    my whole body feels itchy and like its on fire    
4          0  @nationwideclass no, it's not behaving at all....   

                                          clean_text  
0      thats bummer shoulda got david carr third day  
1  upset cant update facebook texting might cry r...  
2  dived many times ball managed save rest go bounds  
3                   whole body feels itchy like fire  
4                           behaving im mad cant see  


___Remove neutral sentiment (value = 2)___<br>
sentiment values: 4 become 1 as positive, 0  as negative <br>
(to binary classification)

In [7]:
df = df[df['sentiment'] != 2]  
df['sentiment'] = df['sentiment'].replace(4, 1) 


Features cleaned text

Labels negative = 0,  positive = 1

Split the data 80, 20

In [8]:
X = df['clean_text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Initialize TF-IDF vectorizer 

fit data

In [9]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [10]:
model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)


LogisticRegression()

accurcy 

 Classification Report

In [11]:
y_pred = model_lr.predict(X_test_tfidf)

print("📊 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))


📊 Accuracy: 0.7734625

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.75      0.77    159494
           1       0.76      0.80      0.78    160506

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



In [12]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train_tfidf, label=y_train)
dtest = xgb.DMatrix(X_test_tfidf, label=y_test)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6
}

xgb_model = xgb.train(params, dtrain, num_boost_round=100)


In [13]:
y_pred_proba = xgb_model.predict(dtest)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_proba]

print("📊 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))


📊 Accuracy: 0.740071875

📄 Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.65      0.71    159494
           1       0.70      0.83      0.76    160506

    accuracy                           0.74    320000
   macro avg       0.75      0.74      0.74    320000
weighted avg       0.75      0.74      0.74    320000



save model

In [14]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(model_lr, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)


In [ ]:
#  streamlit run "C:\Users\RAWAN\OneDrive\Desktop\pipline_internship\task_2\app.py"